In [1]:
# Import necessary libraries:
from intake_esgf import ESGFCatalog
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

In [2]:
# Defines neccessary fucntions:
def Sanderson(skill_scores, d_q):

    # Calculate model weights using the modified Sanderson Approach equation
    weights_raw = [np.exp(skill_score / d_q) for skill_score in skill_scores]
    # Use the built-in sum() function to sum the elements of the list
    weights_sum = sum(weights_raw)
     # Normalize the weights by dividing by the total sum of weights
    weights = [weight / weights_sum for weight in weights_raw]

    return weights


In [3]:
# Extract desired information from ILAMB
df = pd.read_csv("https://www.ilamb.org/CMIP6/historical/scalar_database.csv")
q = df[
    (df.Variable == "Evapotranspiration")
    & (df.ScalarName == "Overall Score")
    & (df.Source == "GLEAMV2.3A")
    & (df.Region == "global")
    & (df.AnalysisType == "MeanState")
]

print(q)

Empty DataFrame
Columns: [Section, Variable, Source, Model, ScalarName, AnalysisType, Region, ScalarType, Units, Data, Weight]
Index: []


In [4]:
# Extract ILAMB skill score from the dictionary above 
source_mean = df[(df.Variable=="Evapotranspiration") & (df.ScalarName=="Overall Score") & (df.Region=="global") & (df.AnalysisType=="MeanState")& (df.Model != "MeanCMIP6")].drop(columns="Weight").groupby('Model').mean(numeric_only=True)
print(source_mean)

# Convert skill scores to numpy array
skill_score_array = np.array(source_mean['Data'])
print(skill_score_array)

                   Data
Model                  
ACCESS-ESM1-5  0.608025
BCC-CSM2-MR    0.648576
CESM2          0.671116
CanESM5        0.607310
GFDL-ESM4      0.661953
IPSL-CM6A-LR   0.654614
MIROC-ESM2L    0.599714
MPI-ESM1.2-HR  0.618888
NorESM2-LM     0.663907
UKESM1-0-LL    0.669652
[0.60802533 0.64857567 0.67111633 0.60731033 0.66195333 0.65461367
 0.59971433 0.61888833 0.663907   0.66965233]


In [5]:
# Now use Sanderson function to estimate weights based on ILAMB skill scores
weights_Sanderson_10 = Sanderson(skill_score_array, 0.1)
print(weights_Sanderson_10)

weights_Sanderson_25 = Sanderson(skill_score_array, 0.25)
print(weights_Sanderson_25)

[0.06985164716286066, 0.10478147596325257, 0.1312735963273362, 0.06935398913322721, 0.11977963347594808, 0.11130308771701311, 0.06428097230568643, 0.07786711086367654, 0.12214273666606339, 0.12936575038493558]
[0.0873542555286281, 0.1027370783026508, 0.11243055627818971, 0.08710477927890463, 0.1083843551627785, 0.10524859116386265, 0.08449799054290784, 0.09123364591245559, 0.10923466087082055, 0.11177408695880153]
